## <span><h1 style= "font-family: garamont; font-size: 40px; font-style: normal; letter-spacing: 5px; background-color: #f6f5f5; color: #fe346e; border-radius: 100px 100px; text-align: center">PREPARATION</h1></span>

In [1]:
%%capture

!pip install nltk rouge-score bert-score sacrebleu

## <span><h1 style= "font-family: garamont; font-size: 40px; font-style: normal; letter-spacing: 5px; background-color: #f6f5f5; color: #fe346e; border-radius: 100px 100px; text-align: center">EVALUATION</h1></span>

In [2]:
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from bert_score import score
import pandas as pd
import sacrebleu
import numpy as np

In [3]:
results_with_gemini = pd.read_csv("/kaggle/input/final-result-with-gemini/results_with_gemini.csv")

In [4]:
import pandas as pd
import math
from collections import Counter
import re


def tokenize(text):
    text = re.sub(r'[^\w\s]', '', text.lower())  # Loại dấu câu, lowercase
    return text.split()

def bleu1(candidates, references):
    bleu = sacrebleu.corpus_bleu(candidates, references, tokenize='13a')
    # BLEU-1: Tính từ precisions[0] (unigram precision) * BP (brevity penalty)
    p1 = bleu.precisions[0] / 100.0  # Unigram precision (0-1)
    bp = bleu.bp  # Brevity penalty (0-1)
    avg_bleu1 = bp * p1
    return avg_bleu1
    
def bleu4(candidates, references):
    bleu = sacrebleu.corpus_bleu(candidates, references, tokenize='13a')
    avg_bleu4 = bleu.score / 100.0
    return avg_bleu4
    
def rouge_l(candidate, reference):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    rouge_l = scores['rougeL'].fmeasure
    return rouge_l

def bert_score(candidate, reference):
    if pd.isna(candidate) or pd.isna(reference) or not candidate.strip() or not reference.strip():
        return 0.0  # Tránh lỗi với text rỗng/NaN
    try:
        # Wrap thành lists cho single pair
        P, R, F1 = score([candidate], [reference], lang="vi", verbose=False)  # verbose=False để sạch output
        bert_f1 = F1.mean().item()  # F1 là harmonic mean của P và R
        return bert_f1
    except Exception as e:
        print(f"Lỗi với pair: {e}")  # Debug nếu cần
        return 0.0

## ===== Tính Bleu-1, 4 ===== ##
candidates = results_with_gemini["generated_answer"].astype(str).to_list()
references = np.array(results_with_gemini["ground_truth"].astype(str).to_list())
references = references.reshape(-1, 1).tolist()

avg_bleu1 = bleu1(candidates, references)
avg_bleu4 = bleu4(candidates, references)

## ===== Tính Rouge_score ===== ##
rouge_scores = [rouge_l(str(row['generated_answer']), str(row['ground_truth'])) for _, row in results_with_gemini.iterrows()]

## ===== Tính Bert_score ===== ##
bert_scores = [bert_score(str(row['generated_answer']), str(row['ground_truth'])) for _, row in results_with_gemini.iterrows()]

# Trung bình
avg_rouge_l = sum(rouge_scores) / len(rouge_scores) if rouge_scores else 0
avg_bert_proxy = sum(bert_scores) / len(bert_scores) if bert_scores else 0

print(f"Số samples: {len(results_with_gemini)}")
print(f"Trung bình BLEU-1: {avg_bleu1:.4f}")
print(f"Trung bình BLEU-4: {avg_bleu4:.4f}")
print(f"Trung bình ROUGE-L: {avg_rouge_l:.4f}")
print(f"Trung bình Proxy BERTScore (Jaccard): {avg_bert_proxy:.4f}")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

...

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Số samples: 1608
Trung bình BLEU-1: 0.6457
Trung bình BLEU-4: 0.2297
Trung bình ROUGE-L: 0.5863
Trung bình Proxy BERTScore (Jaccard): 0.8397
